Import Requirements

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import albumentations as A

Set the Hyperparameters

In [2]:
model_name = "efficientnet_b0"
weights = None #"imagenet"
input_dims = (32, 32, 3)
num_classes = 10
epochs = 25
batch_size = 1024
shuffle_size = 16
learning_rate = 0.01
precision = "float32"
model_dir = "models"

Create the Models

In [3]:
import resnet as resnet

inputs = tf.keras.layers.Input(shape=input_dims)
bb = resnet.ResNet50V2(input_shape=input_dims, num_classes=num_classes, pretrain=False)(inputs)
model = tf.keras.Model(inputs, bb)

In [4]:
"""
odels = {
    'efficientnet_b0': tf.keras.applications.EfficientNetB0,
    'efficientnet_b1': tf.keras.applications.EfficientNetB1,
    'efficientnet_b2': tf.keras.applications.EfficientNetB2,
    'efficientnet_b3': tf.keras.applications.EfficientNetB3,
    'efficientnet_b4': tf.keras.applications.EfficientNetB4,
    'efficientnet_b5': tf.keras.applications.EfficientNetB5,
    'efficientnet_b6': tf.keras.applications.EfficientNetB6,
    'efficientnet_b7': tf.keras.applications.EfficientNetB7
}
inputs = tf.keras.Input(shape=input_dims)
backbone = models[model_name](include_top=True,
                        weights=weights,
                        input_shape=input_dims,
                        classes=num_classes)(inputs)
backbone.trainable = True
# backbone = tf.keras.layers.Dense(num_classes, activation="softmax")(backbone)
model = tf.keras.Model(inputs=inputs, outputs=backbone, name=model_name)

"""

'\nodels = {\n    \'efficientnet_b0\': tf.keras.applications.EfficientNetB0,\n    \'efficientnet_b1\': tf.keras.applications.EfficientNetB1,\n    \'efficientnet_b2\': tf.keras.applications.EfficientNetB2,\n    \'efficientnet_b3\': tf.keras.applications.EfficientNetB3,\n    \'efficientnet_b4\': tf.keras.applications.EfficientNetB4,\n    \'efficientnet_b5\': tf.keras.applications.EfficientNetB5,\n    \'efficientnet_b6\': tf.keras.applications.EfficientNetB6,\n    \'efficientnet_b7\': tf.keras.applications.EfficientNetB7\n}\ninputs = tf.keras.Input(shape=input_dims)\nbackbone = models[model_name](include_top=True,\n                        weights=weights,\n                        input_shape=input_dims,\n                        classes=num_classes)(inputs)\nbackbone.trainable = True\n# backbone = tf.keras.layers.Dense(num_classes, activation="softmax")(backbone)\nmodel = tf.keras.Model(inputs=inputs, outputs=backbone, name=model_name)\n\n'

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 10)                23585290  
                                                                 
Total params: 23,585,290
Trainable params: 23,539,850
Non-trainable params: 45,440
_________________________________________________________________


Import test training data (MNIST)

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


Preprocessing

In [7]:
def augment_preprocess(image):
    image = np.array(image)
    transform = A.Compose([
        A.RandomCrop(width=input_dims[0], height=input_dims[1]),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.Resize(height=input_dims[0], width=input_dims[1]),
    ])
    transformed_image = np.array(transform(image=image)["image"])
    return transformed_image

def one_hot(indicies):
    return tf.one_hot(indicies, num_classes)[0]

In [8]:
x_training_np = np.array([augment_preprocess(image) for image in x_train])
y_training_np = np.array([one_hot(label) for label in y_train])

In [9]:
print("Image shapes" + str(x_training_np.shape))
print("Label shape" + str(y_training_np.shape))

Image shapes(50000, 32, 32, 3)
Label shape(50000, 10)


Training Settings

In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

Training

In [11]:
model.compile(optimizer=optimizer, loss=loss_func, metrics=["accuracy", "categorical_crossentropy"])
model.fit(x_training_np, y_training_np, epochs=epochs)

Epoch 1/25
1563/1563 [==============================] - 85s 48ms/step - loss: 4.7819 - accuracy: 0.2733 - categorical_crossentropy: 2.2100
Epoch 2/25
1563/1563 [==============================] - 76s 48ms/step - loss: 2.0054 - accuracy: 0.3926 - categorical_crossentropy: 1.6315
Epoch 3/25
1563/1563 [==============================] - 75s 48ms/step - loss: 1.7957 - accuracy: 0.4307 - categorical_crossentropy: 1.5372
Epoch 4/25
1563/1563 [==============================] - 76s 48ms/step - loss: 1.6759 - accuracy: 0.4745 - categorical_crossentropy: 1.4346
Epoch 5/25
1563/1563 [==============================] - 75s 48ms/step - loss: 1.5930 - accuracy: 0.5078 - categorical_crossentropy: 1.3514
Epoch 6/25
1563/1563 [==============================] - 76s 49ms/step - loss: 1.5245 - accuracy: 0.5317 - categorical_crossentropy: 1.2977
Epoch 7/25
1563/1563 [==============================] - 76s 48ms/step - loss: 1.4615 - accuracy: 0.5511 - categorical_crossentropy: 1.2505
Epoch 8/25
1563/1563 [=====

Testing

In [ ]:
acc = tf.keras.metrics.CategoricalAccuracy()

x_test_ds = tf.data.Dataset.from_tensor_slices(x_test)
y_test_ds = tf.data.Dataset.from_tensor_slices(y_test)
test_dataset = tf.data.Dataset.zip((x_test_ds, y_test_ds)).batch(batch_size)

for x, y in test_dataset:
    out = model(x, training=False)
    acc.update_state(y_true=y[0], y_pred=out)
print(acc.result())

tf.Tensor(0.1092, shape=(), dtype=float32)
